In [17]:
import json
import requests
import pandas as pd

def get_uniswap_graphql(payload) -> pd.DataFrame:
    try:
        r = requests.post(
            'https://api.thegraph.com/subgraphs/name/1hive/uniswap-v2',
            data=json.dumps(payload),
            timeout=30.0
        )
        print(f"thegraph status_code: {r}")
        r1 = r.json()
        if r1.get('data'):
            return pd.DataFrame(result['data']['pairs'])
        else:
            raise ValueError('no data in response')
    except Exception as e:
        print(f"doh: {e}")
 
graphql_payload_1st1000 = {
    "operationName": "pairs",
    "variables": {},
    "query":"query pairs {\n pairs(first: 1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 \n} \n}\n"
}

df = get_uniswap_graphql(graphql_payload_1st1000)
print(len(df))
# pair_addr: str combining both addresses for the purpose of detecting duplicates pairs.
df['pair_addr'] = [f"{x['id']}/{y['id']}" for x, y in zip(df['token0'], df['token1'])]
# pair_symbol: str combining both symbols for the purpose of detecting duplicates pairs.
df['pair_symbol'] = [f"{x['symbol']}/{y['symbol']}" for x, y in zip(df['token0'], df['token1'])]
df.head()

thegraph status_code: <Response [200]>
882


,id,reserve0,reserve1,token0,token1,pair_addr,pair_symbol
0,0x002b85a23023536395d98e6730f5a5fe8115f08b,0.000127493510283084,0.016367798175231776,"{'decimals': '18', 'id': '0x71850b7e9ee3f13ab4...","{'decimals': '18', 'id': '0x9c374cc5da1d7ee668...",0x71850b7e9ee3f13ab46d67167341e4bdc905eef9/0x9...,HNY/DEXP
1,0x00892cf46a56d78328f03a9cb9361a02abc47511,85488.988743912020283322,1.772824058241431257,"{'decimals': '18', 'id': '0x2f9cebf5de3bc25e06...","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab4...",0x2f9cebf5de3bc25e0643d0e66134e5bf5c48e191/0x7...,xBRICK/HNY
2,0x009d7fb78f2fb1fe6eff81be3136d02faf46db4d,0.0183615627848776,460.18820079,"{'decimals': '18', 'id': '0x71850b7e9ee3f13ab4...","{'decimals': '8', 'id': '0xd7738ec74f08f3a14c4...",0x71850b7e9ee3f13ab46d67167341e4bdc905eef9/0xd...,HNY/DOGE
3,0x0172155c565e783c51b96b608316d489b9bd7463,116.566588612344918231,3.075280774755157096,"{'decimals': '18', 'id': '0x759a891d86436737fc...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",0x759a891d86436737fca1fad68e6d77b1198bb40d/0xe...,TEST/WXDAI
4,0x01f4a4d82a4c1cf12eb2dadc35fd87a14526cc79,47875.192845,48225.689212763644244176,"{'decimals': '6', 'id': '0xddafbb505ad214d7b80...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",0xddafbb505ad214d7b80b1f830fccc89b60fb7a83/0xe...,USDC/WXDAI


In [14]:
df[df.duplicated(['pair_addr'], keep=False)]

,id,reserve0,reserve1,token0,token1,pair_addr,pair_symbol


In [18]:
df_dup = df[df.duplicated(['pair_symbol'], keep=False)]
print(len(df_dup))
df_dup[['id', 'token0', 'token1', 'pair_symbol']].sort_values('pair_symbol',  ascending=False)

96


,id,token0,token1,pair_symbol
370,0x682302b7dff726819b189d5c161aea7ca331b2d4,"{'decimals': '18', 'id': '0x32bc30165b6059cadb...","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab4...",zzz/HNY
596,0xa7967209efb10e42a9d0c6c5b4a585cc44769264,"{'decimals': '18', 'id': '0x15461ca3cfa87b69b6...","{'decimals': '18', 'id': '0x71850b7e9ee3f13ab4...",zzz/HNY
505,0x8f6b98cc0ab15486b344393333c20d988e0e2071,"{'decimals': '18', 'id': '0x10d7116422d3ad15cf...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",xdbx/WXDAI
500,0x8e2a73b43a1bc3093f8c96a41d43c095de79a8a1,"{'decimals': '18', 'id': '0xdaadd8d96d01e47ee5...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",xdbx/WXDAI
407,0x71a5d0a0801544894d94a6056d459c4854ab68de,"{'decimals': '18', 'id': '0xd1a1afd803339e4eaa...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",wTDAO/WXDAI
...,...,...,...,...
62,0x109437f31702f983b798de5032ecb4487422022d,"{'decimals': '18', 'id': '0x43766080c052d97a82...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",BAO/WXDAI
111,0x20f5640639094e4c4e9522113786cf0788f7dcca,"{'decimals': '18', 'id': '0x50dbde932a94b0c23d...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",ALVIN/WXDAI
605,0xa987addeda3724da750c2ade835821e0ea412701,"{'decimals': '18', 'id': '0x3008ff3e688346350b...","{'decimals': '18', 'id': '0xe91d153e0b41518a2c...",ALVIN/WXDAI
20,0x06bd3e59a9ff14f84989d25769839a1486ca3d7d,"{'decimals': '18', 'id': '0x3a97704a1b25f08aa2...","{'decimals': '18', 'id': '0x50dbde932a94b0c23d...",AGVE/ALVIN
